In [2]:
import pandas as pd
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

### Đọc dữ liệu

In [3]:
data = pd.read_csv("datasets/data_preprocess.csv")
data['date'] = pd.to_datetime(data['date'])
data.head()

,Unnamed: 0,film_code,cinema_code,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,0,1492,304,3900000,26,0,4,4.26,150000.0,26,610.328638,2018-05-05,5,2,5
1,1,1492,352,3360000,42,0,5,8.08,80000.0,42,519.801980,2018-05-05,5,2,5
2,2,1492,489,2560000,32,0,4,20.00,80000.0,32,160.000000,2018-05-05,5,2,5
3,3,1492,429,1200000,12,0,1,11.01,100000.0,12,108.991826,2018-05-05,5,2,5
4,4,1492,524,1200000,15,0,3,16.67,80000.0,15,89.982004,2018-05-05,5,2,5


#### Hàm tạo dataframe có index là cột `date` và cột `col_name` được truyền vào

In [4]:
def get_df(col_name, agg='sum'):
    df = data.groupby(['date'])[col_name].agg(agg).reset_index()
    df.set_index('date', inplace=True)
    df = df.asfreq('D')
    df.fillna(df.median(), inplace=True)
    return df

### Phân tích lượng vé bán ra theo thời gian

**Trường dữ liệu**

- `tickets_sold`: lượng vé bán ra

**Lý do chọn biểu đồ**

- Đây là cột dữ liệu chứa thông tin về lượng vé bán ra theo thời gian.
- Bên cạnh lượng doanh thu của các rạp chiếu, dữ liệu về lượng vé bán được đóng vai trò 
không nhỏ vì nó cho thấy được sự thay đổi của lượng khách hàng đến rạp chiếu phim theo thời gian.
- Để phân tích kĩ hơn dữ liệu này, ta có thể phân tách nó ra thành các thành phần mùa vụ bằng `tsa.seasonal_decompose()` và sử dụng các biểu đồ đường.
    - Để phân tách một dữ liệu time series thành các thành phần mùa vụ, ta có 2 mô hình khác nhau có thể sử dụng:
        - Mô hình `additive`: $y_t = T_t + S_t + e_t$
        - Mô hình `multiplicative`: $y_t = T_t \times S_t \times e_t$
    - Trong đó:
        - $y_t$: giá trị của dữ liệu time series tại thời điểm $t$
        - $T_t$: giá trị của thành phần trend tại thời điểm $t$
        - $S_t$: giá trị của thành phần seasonal tại thời điểm $t$
        - $e_t$: giá trị của thành phần residual tại thời điểm $t$

In [5]:
def plotseasonal(res, fig, col):
    fig.add_trace(go.Scatter(x=res.observed.index, y=res.observed), row=1, col=col)
    fig.add_trace(go.Scatter(x=res.trend.index, y=res.trend), row=2, col=col)
    fig.add_trace(go.Scatter(x=res.seasonal.index, y=res.seasonal), row=3, col=col)
    fig.add_trace(go.Scatter(x=res.resid.index, y=res.resid), row=4, col=col)
    

tickets_df = get_df('tickets_sold')
fig = make_subplots(rows=4, cols=2, subplot_titles=("Additive - Dữ liệu gốc", "Multiplicative - Dữ liệu gốc",
                                                    "Additive - Xu hướng", "Multiplicative - Xu hướng",
                                                    "Additive - Chu kì", "Multiplicative - Chu kì",
                                                    "Additive - Nhiễu", "Multiplicative - Nhiễu"),
                    shared_xaxes=True, vertical_spacing=0.1,)

add_res = sm.tsa.seasonal_decompose(tickets_df["tickets_sold"], model="additive")
plotseasonal(add_res, fig, 1)
mul_res = sm.tsa.seasonal_decompose(tickets_df["tickets_sold"], model="multiplicative")
plotseasonal(mul_res, fig, 2)

fig.update_layout(height=1000, width=1000,
                  title_text="Phân tích phân rã chuỗi thời gian dữ liệu vé bán ra", showlegend=False)

#### Nhận xét

- Hai biểu đồ nhiễu cho thấy rằng mô hình multiplicative sẽ phù hợp với dữ liệu tốt hơn mô hình additive, vì vậy chúng ta sẽ sử dụng mô hình nhân để phân tích dữ liệu này.
- Như chúng ta có thể thấy, dữ liệu không phải là dữ liệu tĩnh (stationary) vì có sự thay đổi trong dữ liệu xu hướng.
- Dữ liệu có vẻ có tính chất mùa vụ mạnh, chúng ta sẽ tìm hiểu kĩ hơn về chu kỳ của nó.

### Phân tích chu kỳ của dữ liệu

Chúng ta sẽ tìm xem dữ liệu này có chu kỳ bao nhiêu ngày bằng cách sử dụng hàm `tsa.acf()`.


In [11]:
seasonal = mul_res.seasonal
acf = sm.tsa.acf(seasonal, fft=True, nlags=len(seasonal))
cycle = acf[1:].argmax() + 1
print(f"Chu kì: {cycle} ngày")

Chu kì: 7 ngày


#### Nhận xét chung

- Ta thấy rằng dữ liệu của chúng ta có chu kỳ 7 ngày, điều này rất phổ biến trong ngành điện ảnh,
vì người ta thường xem phim vào một số ngày nhất định trong tuần, và ít xem hơn vào các ngày khác.
- Chúng ta có thể xem xét kỹ hơn để xem ngày nào bán được nhiều vé nhất bằng cách sử dụng biểu đồ cột
thể hiện lượng vé bán ra theo các ngày trong tuần.

In [7]:
grouped_data = tickets_df.groupby(tickets_df.index.day_name()).sum()
grouped_data.index = pd.CategoricalIndex(
    grouped_data.index,
    categories=[
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
    ],
    ordered=True,
)
grouped_data = grouped_data.sort_index()
px.bar(grouped_data, x=grouped_data.index, y="tickets_sold", 
       title="Số vé bán ra theo ngày trong tuần",
       labels={"tickets_sold": "Số vé bán ra", "date": "Ngày trong tuần"})

#### Nhận xét về lượng vé bán theo ngày trong tuần

- Lượng vé bán ra vượt trội hẳn vào ngày thứ 3 (hơn 2/3 so với ngày thứ 6 - ngày có lượng vé bán ra nhiều thứ hai)
- Thường thì những ngày nghỉ như thứ 7 hay chủ nhật mọi người sẽ có thời gian rảnh để đi xem phim hơn là các ngày trong tuần, tuy nhiên ngày thứ 3 (1 ngày bình thường trong tuần) lại có số lượng vé bán ra nhiều nhất, có thể do ngày này có giá vé thấp hơn hoặc có những chương trình ưu đãi khác.
- Để kiểm chứng giả thuyết này, chúng ta sẽ xem xét trung bình giá vé bán ra theo các ngày trong tuần.

In [8]:
sales_df = get_df('ticket_price', agg='mean')
grouped_data = sales_df.groupby(sales_df.index.day_name()).mean()
grouped_data.index = pd.CategoricalIndex(
    grouped_data.index,
    categories=[
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
    ],
    ordered=True,
)
grouped_data = grouped_data.sort_index()
px.bar(grouped_data, x=grouped_data.index, y="ticket_price", 
       title="Giá vé bán ra trung bình theo ngày trong tuần",
       labels={"total_sales": "ticket_price", "date": "Ngày trong tuần"})

#### Nhận xét về trung bình giá vé bán ra theo ngày trong tuần

- Từ biểu đồ ta thấy giá vé vào thứ 3 là thấp, các ngày còn lại cao và không chênh lệch nhau nhiều.
- Đúng như dự đoán thì giá vé vào ngày thứ 3 (xấp xỉ 55,000) chỉ bằng 2/3 các ngày khác
(dao động từ 80,000 đến 85,000). Đây chính là nhân tố ảnh hưởng đến lượng vé bán ra vào ngày thứ 3.

### Phân tích xu hướng vé bán được theo thời gian

Để thực hiện việc này, ta sẽ sử dụng 2 kĩ thuật là Moving Average (`pd.rolling`) và Exponential Weighted Moving Average (`pd.ewm`).

In [15]:
ma_df = tickets_df.copy()
ma_df["MA_7"] = tickets_df["tickets_sold"].rolling(window=7).mean()
ma_df["MA_30"] = tickets_df["tickets_sold"].rolling(window=30).mean()
ma_df["MA_60"] = tickets_df["tickets_sold"].rolling(window=60).mean()
ma_df["EWMA_7"] = tickets_df["tickets_sold"].ewm(span=7).mean()
ma_df["EWMA_30"] = tickets_df["tickets_sold"].ewm(span=30).mean()
ma_df["EWMA_60"] = tickets_df["tickets_sold"].ewm(span=60).mean()

fig = make_subplots(rows=1, cols=2, subplot_titles=("Moving Average", "Exponential Weighted Moving Average"),
                    shared_yaxes=True, vertical_spacing=0.1)

fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df["tickets_sold"], name="Dữ liệu gốc",
                         opacity=0.3), row=1, col='all')

fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df["MA_7"], name="MA_7"), row=1, col=1)
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df["MA_30"], name="MA_30"), row=1, col=1)
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df["MA_60"], name="MA_60"), row=1, col=1)

fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df["EWMA_7"], name="EWMA_7"), row=1, col=2)
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df["EWMA_30"], name="EWMA_30"), row=1, col=2)
fig.add_trace(go.Scatter(x=ma_df.index, y=ma_df["EWMA_60"], name="EWMA_60"), row=1, col=2)

fig.update_layout(height=500, width=1000, title_text="Phân tích dữ liệu vé bán ra")

## Nhận xét

- Chúng ta có thể thấy rằng với cùng một khoảng thời gian, EWMA (Exponential Weighted Moving Average) nhạy hơn (theo sát dữ liệu gốc hơn) so với MA (Moving Average).
Điều này là do EWMA đưa ra trọng số lớn hơn cho dữ liệu gần đây hơn so với MA.
- Biểu đồ cho thấy lượng vé bán ra tăng vọt vào tháng 5, 8 và 11. Biết rằng dataset này có khoảng
thời gian thuộc năm 2018, ta có thể lí giải những sự tăng vọt đó là do các bộ phim bom tấn được ra mắt vào thời điểm đó.
Cụ thể là Avengers: Infinity War (27/4/2018), Mission: Impossible – Fallout (27/7/2018) và 
Fantastic Beasts: The Crimes of Grindelwald (16/11/2018).

references: 
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.asfreq.html
- https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html
- https://plotly.com/python/subplots/
- https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.acf.html#statsmodels.tsa.stattools.acf
- https://pandas.pydata.org/docs/reference/api/pandas.CategoricalIndex.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html
- https://towardsdatascience.com/time-series-analysis-with-statsmodels-12309890539a